# Policy-based method: REINFORCE

Now that we've implemented a value-based algorithm, it's only right that we should try out a policy-based one as well, right ? So let's learn about REINFORCE !

**Key facts**:
- It was first defined in ['Simple Statistical Gradient-Following Algorithms for Connectionist Reinforcement Learning' by Ronald J. WILLIAMS in 1992](https://link.springer.com/content/pdf/10.1007/BF00992696.pdf?pdf=button).
- It uses a **monte carlo** method

### Monte Carlo

As explained in the previous notebook, you can think of Monte Carlo as a method in which our agent learns after each episode instead of doing so at each time step like Temporal Difference.

This implies that there is no need to estimate the target: we can compute the episodic reward for each timestep using the memory batch:

![Monte Carlo formula](./assets/fig13.svg)

Let's begin by implementing this formula !


In [1]:
# Import necessary libraries
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# Set the learning rate and discount factor
lr = 2e-4
gamma = 0.99995

# Set the number of episodes to run
episodes = 400


In [2]:
import gym

# Set the environment to use
env_name = 'CarRacing-v2'
# Create the environment
env = gym.make(env_name, render_mode="rgb_array", continuous=False)
state, _ = env.reset()

In [3]:
device = torch.device("cuda:0")
boost = 1
input_nbr = 96*96
output_nbr = 5
transform = torchvision.transforms.Grayscale(1).to(device)
# Define a neural network to model the policy
class NeuralNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()

        
        self.conv1 = torch.nn.Conv2d(1, 1, 5, padding=2) # Add a conv2d with a kernel size equal to 5
        self.maxpool1 = torch.nn.MaxPool2d(2, stride=2) # Add a MaxPool2d with kernel size and stride equal to 2
        self.conv2 = torch.nn.Conv2d(1, 1, 5, padding=2)# Add a conv2d with a kernel size equal to 5
        self.maxpool2 = torch.nn.MaxPool2d(2, stride=2) # Add a MaxPool2d with kernel size a
        
        # Create fully-connected layers with ReLU activations
        self.fc1 = torch.nn.Linear(24*24, 16*boost).to(device)
        self.fc2 = torch.nn.Linear(16*boost, output_nbr).to(device)
        self.softmax = torch.nn.Softmax(dim=0).to(device)

        self.actions, self.states, self.rewards = [], [], []
        
    def forward(self, x):
        x = transform(torch.tensor(x, dtype=torch.float32).permute(2, 0, 1)).reshape((1, 1, 96, 96)).to(device)
        x = torch.nn.functional.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = torch.nn.functional.relu(self.conv2(x))
        x = self.maxpool2(x)
        
        
        x = x.reshape((24*24))
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        
        return self.softmax(x)

network = NeuralNetwork(env).to(device)

# Use Adam optimizer to optimize the neural network
optim = torch.optim.Adam(network.parameters())

In [4]:
# Import the deque class from the collections module
from collections import deque

# Initialize empty lists for rewards and losses
recent_rewards = deque(maxlen=100)
train_rewards = []
train_loss = []

# We will avoid rendering our environment during training: 
# it would tremendously slow down the process
env = gym.make(env_name, continuous=False) 

In [5]:
from torch.distributions import Categorical

def policy_action(self, state):
    # Get the probabilities for each action, using the current state
    x = self.forward(state)
    # Create a distribution according to the probabilities
    m = Categorical(x)
    # Sample an action from the distribution
    action = m.sample()
    # Return the chosen action
    return action.item()

NeuralNetwork.policy_action = policy_action

In [6]:
def remember(self, Action, State, Reward):
    self.actions.append(Action)
    self.states.append(State)
    self.rewards.append(Reward)
    
NeuralNetwork.remember = remember

In [7]:
def discount_rewards(self):
    ## Discount the returns using the discount factor
    return np.cumsum(np.array(self.rewards[::-1])*gamma)[::-1]

NeuralNetwork.discount_rewards = discount_rewards

network.rewards = [0.2, 0.6, 0.1, 1.2, 0.9]
network.discount_rewards()

array([2.99985 , 2.79986 , 2.19989 , 2.099895, 0.899955])

Finally, let's implement our gradient ascent !

Since Adam, our optimizer, does most of the job for us, we only need to provide it the correct loss.\
In REINFORCE, the loss is defined as follows:

$$ L = G \nabla \ln \pi(A_t|S_t,\theta) $$

The $ \nabla $ symbol represents the gradient, so you can understand this formula as: `loss = G * log_prob` 

Because we are attempting to find the parameters of our policy which **maximize** the expected cumulative reward, we will not use the familiar **gradient descent** and instead use what is known as **gradient ascent**.

Fear not, because it is not very complicated !

With gradient descent, we update the parameters in the opposite direction of the gradient, which decreases the training cost or the expected cumulative reward, in this case.

So in order to use gradient **ascent**, we must aim to **increase** the expected cumulative reward. Which means we 
can achieve this by simply doing a `backward()` on a negative loss !

This leads us back to the Monte Carlo formula for value-based methods:

$$ G_t - V(S_t) $$

or, for policy-based methods:

$$ G[- \nabla \ln \pi(A_t|S_t,\theta)] $$

In [8]:
def gradient_ascent(self, discounted_rewards):
    # Perform gradient ascent to update the probabilities in the distribution
    for State, Action, G in zip(self.states, self.actions, discounted_rewards):
        # Get the probabilities for the current state
        probs = network.forward(State)
        
        # Calculate the loss as the negative log probability of the chosen action
        # multiplied by the discounted return
        loss = G * (-Categorical(probs).log_prob(torch.tensor(Action).to(device)))

        # Clear the gradients, backpropagate the loss, and update the network parameters
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        #

NeuralNetwork.gradient_ascent = gradient_ascent

In [9]:
# Iterate over the number of episodes
import time

for episode in range(episodes):
    # Reset the environment and initialize empty lists for actions, states, and rewards
    state, _  = env.reset()
    network.actions, network.states, network.rewards = [], [], []

    # Train the agent for a single episode
    
    for _ in range(100):
        action = network.policy_action(state)
        # Take the action in the environment and get the new state, reward, and done flag
        new_state, reward, termination, truncation, _ = env.step(action)
    
        # Save the action, state, and reward for later
        network.remember(action, state, reward)

        state = new_state

        # If the episode is done or the time limit is reached, stop training
        if termination or truncation:
            break
    # Perform gradient ascent
    network.gradient_ascent(network.discount_rewards())

    # Save the total reward for the episode and append it to the recent rewards queue
    train_rewards.append(np.sum(network.rewards))
    recent_rewards.append(train_rewards[-1])
    
    # Print the mean recent reward every 50 episodes
    if episode % 50 == 0:
        print(f"E:{episode} R:{np.mean(recent_rewards):>6.3f}")

    if np.mean(recent_rewards) > 400:
        break

/home/alexandre/.local/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


E:0 R:36.110


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots()

ax.plot(train_rewards)
ax.plot(gaussian_filter1d(train_rewards, sigma=20), linewidth=4)
ax.set_title('Rewards')

fig.show()
print(f"{1/10**5:.7f}")

Finally, let's display five episodes of our trained agent to see how glorious it is:

In [ ]:
env = gym.make(env_name, render_mode="human")

for _ in range(5):
    Rewards = []
    
    state, _ = env.reset()
    done = False
    
    for _ in range(1000):
        # Calculate the probabilities of taking each action using the trained
        # neural network
        probs = network.forward(state)
        
        # Sample an action from the resulting distribution using the 
        # torch.distributions.Categorical() method
        action = int(torch.distributions.Categorical(probs).sample())

        new_state, reward, termination, truncation, _ = env.step(action)
    
        state = new_state

        Rewards.append(reward)

        if termination or truncation:
            break
    
    # Print the total rewards for the current episode
    print(f'Reward: {sum(Rewards)}')

# Close the environment
env.close()

Reward: 41.0
Reward: 34.0
Reward: 27.0
Reward: 26.0
Reward: 29.0


Wow, you really did it ! You've succesfully implemented both a value-based and a policy-based method in reinforcement learning ! And, to top it all off, you even managed to solve CartPole using OpenAI Gym !

If you're up for it, let's head over to section 2 and go **deeper** within the field of RL by returning to value-based methods and implementing the successor to Q-Learning, Deep Q Network, or DQN for short !

Good luck !